In [ ]:
!pip install cartopy #Install

# Creating the graph

code was modified and adapted from https://github.com/Data-Monkey/GTFS-NetworkX/blob/master/GTFStoGraph.py


In [ ]:
"""
Created on Tue May  8 17:48:55 2018

@author: roland

inspired by https://github.com/paulgb/gtfs-gexf/blob/master/transform.py
"""


#Import Packages
import networkx as nx
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from csv import DictReader
from itertools import groupby
import cartopy.feature as cfeature

#Create file for storage
TRIPS_FILE = 'trips.txt'
ROUTES_FILE = 'routes.txt'
STOPS_FILE = 'stops.txt'

INCLUDE_AGENCIES=['VTA']

IGNORE_ROUTE=[]

G = nx.MultiGraph()

def get_stop_id(stop_id):
    """ translate stop_id to parent_stop_id
        if available
    """
    if STOPS[stop_id]['parent_station'] == '':
        return stop_id
    else:
        return STOPS[stop_id]['parent_station']


def add_stop_to_graph(G, stop_id):
    """ add stop as new node to graph
    """
    #lookup details of the stop (parent stop if available)
    node = STOPS[get_stop_id(stop_id)]

    if node['stop_id'] not in G.nodes:
        G.add_node(node['stop_id'],
                   stop_name = node['stop_name'],
                   lon = float(node['stop_lon']),
                   lat = float(node['stop_lat']))
    return G

def add_edge_to_graph(G, from_id, to_id, route_long_name):
    """ add edge to graph
        adding the route long name as a key
        if the edge and key exist, increment the count
    """
    edge = G.get_edge_data(get_stop_id(from_id), get_stop_id(to_id),route_long_name, default = 0)
    if edge == 0:
        G.add_edge(get_stop_id(from_id), get_stop_id(to_id),
                   key=route_long_name,
                   count=1)
    else:
        G.add_edge(get_stop_id(from_id), get_stop_id(to_id),
                   key=route_long_name,
                   count=edge['count']+1)


def load_routes(filename):
    """ include only routes from agencies we are interested in
    """
    routes_csv = DictReader(open(filename,'r'))
    routes_dict = dict()
    for route in routes_csv:
        if (route['agency_id'] in INCLUDE_AGENCIES and
            route['route_id'] not in IGNORE_ROUTE ):

            routes_dict[route['route_id']] = route
    print ('routes', len(routes_dict))
    return routes_dict


def load_trips(filename, routes_dict):
    """ load trips from file
        only include trips on routes we are interested in
    """
    trips_csv = DictReader(open(filename,'r'))
    trips_dict = dict()
    for trip in trips_csv:
        if trip['route_id'] in routes_dict:
            trip['color'] = routes_dict[trip['route_id']]['route_color']
            trip['route_long_name']=routes_dict[trip['route_id']]['route_long_name']
            trips_dict[trip['trip_id']] = trip
    print ('trips', len(trips_dict))
    return trips_dict


def load_stops(filename):
    stops_csv = DictReader(open(filename,'r'))
    stops_dict = dict()
    for stop in stops_csv:
        stops_dict[stop['stop_id']] = stop
    print ('stops', len(stops_dict))
    return stops_dict


# ==============================================

#Load Routes, Trips, Stops to storage file
ROUTES = load_routes(filename=ROUTES_FILE)
TRIPS = load_trips(filename=TRIPS_FILE, routes_dict=ROUTES)
STOPS = load_stops(filename=STOPS_FILE)

stop_times_csv = DictReader(open('stop_times.txt','r'))

stops = set()
edges = dict()
for trip_id, stop_time_iter in groupby(stop_times_csv, lambda stop_time: stop_time['trip_id']):
    if trip_id in TRIPS:
        trip = TRIPS[trip_id]
        prev_stop = next(stop_time_iter)['stop_id']
        stops.add(prev_stop)
        for stop_time in stop_time_iter:
            stop = stop_time['stop_id']
            edge = (prev_stop, stop)
            edges[edge] = trip['route_long_name']
            stops.add(stop)
            prev_stop = stop
print ('stops', len(stops))
print ('edges', len(edges))

for stop_id in STOPS:
    if stop_id in stops:
       add_stop_to_graph(G, stop_id)
print('Nodes:', G.number_of_nodes() )

for (start_stop_id, end_stop_id), route_long_name in edges.items():
    add_edge_to_graph(G,
                      from_id = start_stop_id,
                      to_id = end_stop_id,
                      route_long_name=route_long_name)
print('Edges:', G.number_of_edges() )

deg = nx.degree(G)
labels = {stop_id: G.nodes[stop_id]['stop_name'] if deg[stop_id] >= 0 else ''
          for stop_id in G.nodes}

pos = {stop_id: (G.nodes[stop_id]['lon'], G.nodes[stop_id]['lat'])
       for stop_id in G.nodes}

nx.write_gml(G, "gtfs_vta_network_data.gml")
print("Graph written to gtfs_vta_network.gml")

plt.show()

routes 76
trips 8612
stops 3355
stops 3277
edges 3706
Nodes: 3158
Edges: 3656
Graph written to gtfs_vta_network.gml


# Getting the fully connected componenents

In [ ]:
#Check if the graph is fully connected
print(nx.is_connected(G))
print(nx.number_connected_components(G))

#print out the connected components
components = (nx.connected_components(G))

#make a graph out of each connected component
graphs = [G.subgraph(c) for c in components]

#write each graph to gml
for i, g in enumerate(graphs):
    nx.write_gml(g, f"graph_connected_components_{i}.gml")


False
2


# Computing centrality measures

In [ ]:
#Computing betweeness, degree and closeness centrailtiy of G
betweenness = nx.betweenness_centrality(G)
degree = nx.degree_centrality(G)
closeness = nx.closeness_centrality(G)

# Calculate maximum nodes and values for each centrality
max_betweenness_node = max(betweenness, key=betweenness.get)
max_betweenness_value = betweenness[max_betweenness_node]

max_degree_node = max(degree, key=degree.get)
max_degree_value = degree[max_degree_node]

max_closeness_node = max(closeness, key=closeness.get)
max_closeness_value = closeness[max_closeness_node]

# Print the results
print("Betweenness centrality:", max_betweenness_node, max_betweenness_value)
print("Degree centrality:", max_degree_node, max_degree_value)
print("Closeness centrality:", max_closeness_node, max_closeness_value)

Betweenness centrality: 811 0.2539401752685307
Degree centrality: PS_MBRT 0.008552423186569527
Closeness centrality: 811 0.06868463551469987


In [ ]:
bridges = list(nx.bridges(G))
print("Bridges:", bridges)

# Access edge attributes for each bridge
for edge in bridges:
    u, v = edge  # Unpack the nodes of the edge
    attributes = G[u][v]  # Access the attributes of the edge
    print(f"Attributes of edge {edge}: {attributes}")

Bridges: [('336', '4440'), ('413', '5515'), ('439', '5516'), ('611', '5463'), ('613', '614'), ('614', '5464'), ('1123', '3694'), ('1904', '5473'), ('1904', '1905'), ('1920', '5726'), ('1920', '1921'), ('1921', '1922'), ('1922', '5473'), ('1972', '4553'), ('3628', '3629'), ('3629', '4657'), ('3630', '4657'), ('3630', '3631'), ('3631', '3632'), ('3632', '3633'), ('3633', '3634'), ('3634', '3635'), ('3635', '3636'), ('3636', '3637'), ('3637', '3638'), ('3656', '3657'), ('3657', '3658'), ('3658', '3659'), ('3659', '3660'), ('3660', '3661'), ('3661', '3662'), ('3662', '3663'), ('3663', '3664'), ('3664', '3665'), ('3665', '3666'), ('3666', '3667'), ('4117', '4181'), ('4118', '5723'), ('4241', '4242'), ('4242', '4243'), ('4332', '4333'), ('4333', '4334'), ('4334', '4335'), ('4440', '4441'), ('4441', '4442'), ('4443', '4444'), ('4444', '4445'), ('4445', '4446'), ('4553', '5060'), ('4555', '5060'), ('4555', '4556'), ('4556', '4557'), ('PS_OHLO', 'PS_BLSM'), ('PS_OHLO', 'PS_BRNM'), ('PS_TRSA', '

# Random Node removal

In [ ]:
#Import packages
import random

def remove_random_nodes_and_calculate(graph, percentage_to_remove=0.1):

    random.seed(42)

    # Create a copy of the graph to preserve the original
    graph_copy = graph.copy()

    # Calculate the number of nodes to remove (ensuring at least one is removed)
    num_nodes_to_remove = max(1, int(len(graph_copy.nodes) * percentage_to_remove))

    # Randomly select nodes to remove
    nodes_to_remove = random.sample(list(graph_copy.nodes), num_nodes_to_remove)

    # Print the nodes being removed
    print(f"Nodes removed: {nodes_to_remove}")

    # Remove the selected nodes from the graph
    graph_copy.remove_nodes_from(nodes_to_remove)

    # Print the current graph state
    print(f"Graph now has {graph_copy.number_of_nodes()} nodes and {graph_copy.number_of_edges()} edges.")

    # Perform the calculation using the provided function
    calculation(graph_copy)

remove_random_nodes_and_calculate(G)

Nodes removed: ['4422', '722', 'PS_GILR', '5797', '1889', '1587', '1391', '873', '5752', '673', 'PS_BORR', '5781', '3620', '593', '3915', '2852', '293', '272', '626', '1354', '1506', '3419', '4023', '245', '3694', '1207', '5595', '4485', '5427', '3619', '2835', '1372', '2997', '3910', '1908', '53', '5962', '968', 'PS_WITC', '2856', '2368', '1907', '949', '1331', '6001', '2345', '672', '621', '2601', '646', '2497', '2390', '4043', '1751', '371', '5695', '3073', '3574', '798', '2589', '552', '3655', '2002', '4335', '4190', '2509', '3782', '1155', '5503', '496', '381', '4639', '1435', '1965', '556', '1507', '666', '3018', '4388', '2522', '982', '2545', '2468', '1285', 'PS_TAMN', '1818', '5459', '5006', '4466', '504', '4104', '4385', '1025', '3565', '5687', '1586', '985', '3088', '2599', '1832', '4424', '5059', '3675', '1366', '5029', '2246', '6054', '438', '1464', '294', '2184', '2743', '1821', '483', '1297', '3727', '5608', '2180', '1303', '4553', '3390', '2716', '4437', '3070', '890', '

gets the metrics for a given graph

In [ ]:
def calculation(g):
    if nx.is_directed(g):
        # Use weakly connected components for directed graphs
        largest_component = max(nx.weakly_connected_components(g), key=len)
    else:
        # Use connected components for undirected graphs
        largest_component = max(nx.connected_components(g), key=len)

    # Create a subgraph for the largest component
    largest_component_subgraph = g.subgraph(largest_component)

    # Calculate the metrics
    average_shortest_path_length = nx.average_shortest_path_length(largest_component_subgraph)
    connectivity = nx.node_connectivity(g)
    global_efficiency = nx.global_efficiency(g)
    diameter = nx.diameter(largest_component_subgraph)

    # Convert to simple graph before calculating clustering coefficient
    g_simple = nx.Graph(g)  # Create a simple graph from the multigraph
    clustering_coefficient = nx.average_clustering(g_simple)  # Calculate on the simple graph

    # Print the results
    print(f"Average Shortest Path Length: {average_shortest_path_length}")
    print(f"Node Connectivity: {connectivity}")
    print(f"Size of Largest Connected Component: {len(largest_component)}")
    print(f"Global Efficiency: {global_efficiency}")
    print(f"Diameter: {diameter}")
    print(f"Clustering Coefficient: {clustering_coefficient}")



In [ ]:
def get_top_betweenness_nodes(graph, percentage_to_remove=0.1):

    # Calculate the number of nodes to remove based on the percentage
    num_nodes_to_remove = max(1, int(len(graph.nodes) * percentage_to_remove))  # Ensure at least 1 node is removed

    # Create a copy of the graph to preserve the original
    graph_copy = graph.copy()

    # Calculate betweenness centrality for all nodes in the graph
    betweenness = nx.betweenness_centrality(graph_copy)

    # Sort nodes by their betweenness centrality, and get the top ones
    top_nodes_betweenness = sorted(betweenness.items(), key=lambda item: item[1], reverse=True)[:num_nodes_to_remove]

    # Extract the node names from the sorted list of betweenness centrality
    top_nodes_betweenness = [node[0] for node in top_nodes_betweenness]

    # Print the nodes being removed
    print(f"Nodes to remove based on betweenness centrality: {top_nodes_betweenness}")

    return top_nodes_betweenness

In [ ]:
def get_top_closeness_nodes(graph, percentage_to_remove=0.1):

    # Calculate the number of nodes to remove based on the percentage
    num_nodes_to_remove = max(1, int(len(graph.nodes) * percentage_to_remove))  # Ensure at least 1 node is removed

    # Create a copy of the graph to preserve the original
    graph_copy = graph.copy()

    # Calculate closeness centrality for all nodes in the graph
    closeness = nx.closeness_centrality(graph_copy)

    # Sort nodes by their closeness centrality, and get the top ones
    top_nodes_closeness = sorted(closeness.items(), key=lambda item: item[1], reverse=True)[:num_nodes_to_remove]

    # Extract the node names from the sorted list of closeness centrality
    top_nodes_closeness = [node[0] for node in top_nodes_closeness]

    # Print the nodes being removed
    print(f"Nodes to remove based on closeness centrality: {top_nodes_closeness}")

    return top_nodes_closeness

In [ ]:
def get_top_degree_nodes(graph, percentage_to_remove=0.1):

    # Calculate the number of nodes to remove based on the percentage
    num_nodes_to_remove = max(1, int(len(graph.nodes) * percentage_to_remove))  # Ensure at least 1 node is removed

    # Create a copy of the graph to preserve the original
    graph_copy = graph.copy()

    # Calculate degree centrality for all nodes in the graph
    degree_centrality = nx.degree_centrality(graph_copy)

    # Sort nodes by their degree centrality, and get the top ones
    top_nodes_degree = sorted(degree_centrality.items(), key=lambda item: item[1], reverse=True)[:num_nodes_to_remove]

    # Extract the node names from the sorted list of degree centrality
    top_nodes_degree = [node[0] for node in top_nodes_degree]

    # Print the nodes being removed
    print(f"Nodes to remove based on degree centrality: {top_nodes_degree}")

    return top_nodes_degree

# Targeted Removal for Degree

In [ ]:
def targeted_node_removal_and_calculate(graph, percentage_to_remove=0.1):

    # Extract node names from the list of tuples
    top_nodes = get_top_degree_nodes(graph, percentage_to_remove)

    # Create a copy of the graph to preserve the original
    graph_copy = graph.copy()

    # Remove the selected nodes from the graph
    graph_copy.remove_nodes_from(top_nodes)

    # Print the new number of nodes and edges
    print(f"Graph now has {graph_copy.number_of_nodes()} nodes and {graph_copy.number_of_edges()} edges.")

    # Call the calculation function to get the robustness metrics
    calculation(graph_copy)

targeted_node_removal_and_calculate(G)

Nodes to remove based on degree centrality: ['PS_MBRT', 'PS_ERTC', 'PS_SCTC', '4692', 'PS_BERR', 'PS_DRDN', '439', 'PS_MVCT', 'PS_GILR', 'PS_PATC', '413', 'PS_SNLS', 'PS_ARTC', 'PS_WITC', '410', '443', '3926', 'PS_LMTC', '4585', 'PS_OHLN', '335', '391', '461', '504', '565', '574', '660', '750', '811', '1124', '1715', '1716', '1782', '2523', '3358', '3417', '3468', '4338', '4578', 'PS_BAYP', 'PS_WVTC', '20', '408', '412', '494', '608', '619', '653', '661', '864', '995', '1123', '1128', '1933', '1962', '2526', '2535', '2604', '3202', '3271', '3371', '3379', '3446', '3456', '3582', '3844', '4507', 'PS_GISH', 'PS_TASM', 'PS_CHMP', '5707', '8', '166', '216', '224', '266', '330', '336', '343', '346', '347', '355', '361', '366', '368', '370', '373', '378', '382', '387', '400', '403', '418', '420', '431', '434', '444', '445', '449', '452', '464', '468', '472', '477', '480', '482', '484', '489', '500', '511', '512', '517', '567', '571', '576', '582', '586', '594', '600', '603', '607', '624', '6

#Targeted Removal for Betweenness Centrality

In [ ]:
def betweenness_node_removal_and_calculate(graph, percentage_to_remove=0.1):

    # Create a copy of the graph to preserve the original
    graph_copy = graph.copy()

    # Sort nodes by their betweenness centrality, and get the top ones
    top_nodes_betweenness = get_top_betweenness_nodes(graph_copy, percentage_to_remove)

    # Print the nodes being removed
    print(f"Nodes to remove based on betweenness centrality: {top_nodes_betweenness}")

    # Remove the selected nodes from the graph
    graph_copy.remove_nodes_from(top_nodes_betweenness)

    # Print the new number of nodes and edges
    print(f"Graph now has {graph_copy.number_of_nodes()} nodes and {graph_copy.number_of_edges()} edges.")

    # Call the calculation function to get the robustness metrics
    calculation(graph_copy)

betweenness_node_removal_and_calculate(G)

Nodes to remove based on betweenness centrality: ['811', 'PS_MBRT', 'PS_ARTC', 'PS_DRDN', '4457', 'PS_SNLS', '4451', '4579', '4434', '3860', '5325', '3853', '3849', '2667', '4692', '5407', '910', '6067', '6048', 'PS_SCTC', '653', 'PS_WITC', 'PS_ERTC', '3417', '1599', '3844', '1110', 'PS_OHLN', '3446', '3821', '736', '4507', '166', '403', '400', '408', '574', '3843', '4672', '4671', '6061', '2604', '1123', '1380', '410', '1377', '660', '1366', '656', '1392', '3443', '4636', '439', '4458', '2535', '571', '1118', '567', '1119', '413', '5797', '565', '216', '504', '335', '4646', '3379', '5513', 'PS_BERR', '112', '113', '1716', '494', '3456', '816', '1715', '2059', '995', '2214', '4433', '1933', '2212', '4435', 'PS_MVCT', '4436', '4342', '224', '157', '4391', '3410', '2126', '4340', '3468', '4389', '443', '4390', '156', '4341', '155', '347', '2162', '2394', '2165', '911', '2524', '855', '912', '913', '3279', '3202', '3704', '3702', '6060', '4097', 'PS_GILR', '915', '617', '916', '917', '211

#Targeted Removal for Closeness Centrality

In [ ]:
def closeness_node_removal_and_calculate(graph, percentage_to_remove=0.1):

    # Create a copy of the graph to preserve the original
    graph_copy = graph.copy()

    # Sort nodes by their closeness centrality, and get the top ones
    top_nodes_closeness = get_top_closeness_nodes(graph_copy, percentage_to_remove)

    # Print the nodes being removed
    print(f"Nodes to remove based on closeness centrality: {top_nodes_closeness}")

    # Remove the selected nodes from the graph
    graph_copy.remove_nodes_from(top_nodes_closeness)

    # Print the new number of nodes and edges
    print(f"Graph now has {graph_copy.number_of_nodes()} nodes and {graph_copy.number_of_edges()} edges.")

    # Call the calculation function to get the robustness metrics
    calculation(graph_copy)

closeness_node_removal_and_calculate(G)

Nodes to remove based on closeness centrality: ['811', '4451', '4579', '5407', '4457', 'PS_ARTC', '4434', 'PS_DRDN', '3821', '6067', '736', 'PS_SNLS', '3860', 'PS_ERTC', '812', '5696', '4636', '408', '410', '4450', '1884', '443', '1599', '5325', '5858', '409', '5859', '4581', '444', '4435', '3038', '3864', '3065', '446', '406', '1123', '6048', '816', '403', '3853', '5513', '2951', '574', '3073', '3024', '737', '445', '653', '1110', '735', '5514', '3188', '3292', '411', '3849', '5850', '3185', '440', '3295', '813', '3186', '3294', '5857', '3907', '3906', '53', '5856', '5707', '1231', '1264', '1229', '1862', '3817', '808', '4449', '448', '2928', '404', '400', '1119', '2667', '4436', '439', '413', '4684', '449', '442', '617', '608', 'PS_MBRT', '431', '910', '420', '814', '571', 'PS_SCTC', '619', '3040', '412', '3291', '3063', '3031', '3066', '434', '1124', '4452', '656', '5656', '1118', '418', '3694', '4433', '2957', '815', '820', '6033', '3189', '402', '576', '452', '3271', '3207', 'PS_O

#Adding Nodes to the orginal graph based on betweenness

In [ ]:
import networkx as nx
#create a copy of exisitng graph
G_adding_nodes = G.copy()
G_orig = G.copy()

top_nodes_betweenness = get_top_betweenness_nodes(G_orig)

#add nodes based on betweenness
for removed_node in top_nodes_betweenness:
    neighbors = list(G_orig.neighbors(removed_node))
    new_node = f"redundant_{removed_node}"
    if G_adding_nodes.has_node(new_node):
        print(f"Duplicate node being added: {new_node}")
    G_adding_nodes.add_node(new_node)
    for neighbor in neighbors:
      if not G_adding_nodes.has_edge(new_node, neighbor):
        G_adding_nodes.add_edge(new_node, neighbor)

print(f"Graph now has {G_adding_nodes.number_of_nodes()} nodes and {G_adding_nodes.number_of_edges()} edges.")

calculation(G_adding_nodes)


Nodes to remove based on betweenness centrality: ['811', 'PS_MBRT', 'PS_ARTC', 'PS_DRDN', '4457', 'PS_SNLS', '4451', '4579', '4434', '3860', '5325', '3853', '3849', '2667', '4692', '5407', '910', '6067', '6048', 'PS_SCTC', '653', 'PS_WITC', 'PS_ERTC', '3417', '1599', '3844', '1110', 'PS_OHLN', '3446', '3821', '736', '4507', '166', '403', '400', '408', '574', '3843', '4672', '4671', '6061', '2604', '1123', '1380', '410', '1377', '660', '1366', '656', '1392', '3443', '4636', '439', '4458', '2535', '571', '1118', '567', '1119', '413', '5797', '565', '216', '504', '335', '4646', '3379', '5513', 'PS_BERR', '112', '113', '1716', '494', '3456', '816', '1715', '2059', '995', '2214', '4433', '1933', '2212', '4435', 'PS_MVCT', '4436', '4342', '224', '157', '4391', '3410', '2126', '4340', '3468', '4389', '443', '4390', '156', '4341', '155', '347', '2162', '2394', '2165', '911', '2524', '855', '912', '913', '3279', '3202', '3704', '3702', '6060', '4097', 'PS_GILR', '915', '617', '916', '917', '211

#Adding Nodes to the graph based on closeness

In [ ]:
import networkx as nx
#clone the exisitng graph
G_adding_nodes_closeness = G.copy()
G_orig = G.copy()
#add nodes based on closeness
top_nodes_closeness = get_top_closeness_nodes(G_orig)
for removed_node in top_nodes_closeness:
    neighbors = list(G_orig.neighbors(removed_node))
    new_node = f"redundant_{removed_node}"
    if G_adding_nodes_closeness.has_node(new_node):
        print(f"Duplicate node being added: {new_node}")
    G_adding_nodes_closeness.add_node(new_node)
    for neighbor in neighbors:
        G_adding_nodes_closeness.add_edge(new_node, neighbor)

print(f"Graph now has {G_adding_nodes_closeness.number_of_nodes()} nodes and {G_adding_nodes_closeness.number_of_edges()} edges.")


calculation(G_adding_nodes_closeness)

Nodes to remove based on closeness centrality: ['811', '4451', '4579', '5407', '4457', 'PS_ARTC', '4434', 'PS_DRDN', '3821', '6067', '736', 'PS_SNLS', '3860', 'PS_ERTC', '812', '5696', '4636', '408', '410', '4450', '1884', '443', '1599', '5325', '5858', '409', '5859', '4581', '444', '4435', '3038', '3864', '3065', '446', '406', '1123', '6048', '816', '403', '3853', '5513', '2951', '574', '3073', '3024', '737', '445', '653', '1110', '735', '5514', '3188', '3292', '411', '3849', '5850', '3185', '440', '3295', '813', '3186', '3294', '5857', '3907', '3906', '53', '5856', '5707', '1231', '1264', '1229', '1862', '3817', '808', '4449', '448', '2928', '404', '400', '1119', '2667', '4436', '439', '413', '4684', '449', '442', '617', '608', 'PS_MBRT', '431', '910', '420', '814', '571', 'PS_SCTC', '619', '3040', '412', '3291', '3063', '3031', '3066', '434', '1124', '4452', '656', '5656', '1118', '418', '3694', '4433', '2957', '815', '820', '6033', '3189', '402', '576', '452', '3271', '3207', 'PS_O

#Adding Nodes to the graph based on Degree Centrality

In [ ]:
import networkx as nx
#clone exisiting graph
G_adding_nodes_degree = G.copy()
G_orig = G.copy()

#add nodes based on degree centrality
top_nodes_degree = get_top_degree_nodes(G_orig)
for removed_node in top_nodes_degree:
    neighbors = list(G_orig.neighbors(removed_node))
    new_node = f"redundant_{removed_node}"
    if G_adding_nodes_degree.has_node(new_node):
        print(f"Duplicate node being added: {new_node}")
    G_adding_nodes_degree.add_node(new_node)
    for neighbor in neighbors:
        G_adding_nodes_degree.add_edge(new_node, neighbor)

print(f"Graph now has {G_adding_nodes_degree.number_of_nodes()} nodes and {G_adding_nodes_degree.number_of_edges()} edges.")

calculation(G_adding_nodes_degree)

Nodes to remove based on degree centrality: ['PS_MBRT', 'PS_ERTC', 'PS_SCTC', '4692', 'PS_BERR', 'PS_DRDN', '439', 'PS_MVCT', 'PS_GILR', 'PS_PATC', '413', 'PS_SNLS', 'PS_ARTC', 'PS_WITC', '410', '443', '3926', 'PS_LMTC', '4585', 'PS_OHLN', '335', '391', '461', '504', '565', '574', '660', '750', '811', '1124', '1715', '1716', '1782', '2523', '3358', '3417', '3468', '4338', '4578', 'PS_BAYP', 'PS_WVTC', '20', '408', '412', '494', '608', '619', '653', '661', '864', '995', '1123', '1128', '1933', '1962', '2526', '2535', '2604', '3202', '3271', '3371', '3379', '3446', '3456', '3582', '3844', '4507', 'PS_GISH', 'PS_TASM', 'PS_CHMP', '5707', '8', '166', '216', '224', '266', '330', '336', '343', '346', '347', '355', '361', '366', '368', '370', '373', '378', '382', '387', '400', '403', '418', '420', '431', '434', '444', '445', '449', '452', '464', '468', '472', '477', '480', '482', '484', '489', '500', '511', '512', '517', '567', '571', '576', '582', '586', '594', '600', '603', '607', '624', '6

In [ ]:
betweenness_node_removal_and_calculate(G_adding_nodes)

Nodes to remove based on betweenness centrality: ['811', 'PS_ARTC', 'PS_MBRT', 'PS_DRDN', '4457', 'PS_SNLS', '4451', '4434', '4579', '3860', '5407', '5325', '3853', '3849', '2667', '4692', '910', '6067', '6048', 'redundant_811', '653', 'PS_SCTC', 'PS_WITC', '3417', '1110', '1599', 'PS_OHLN', '3844', '3446', '4507', '400', '3821', '408', '1123', '403', '736', '3843', '166', '574', '4672', '6061', '4671', '2604', '1380', '1377', '410', 'redundant_PS_ARTC', 'redundant_PS_MBRT', 'PS_ERTC', 'redundant_PS_DRDN', 'redundant_PS_SNLS', '4636', '656', '1366', '3443', 'redundant_4457', '660', '1392', '1119', 'redundant_4451', '1118', '5797', '4458', '571', 'redundant_4434', '2535', '439', '567', '112', '113', '504', 'redundant_4579', 'redundant_3860', 'redundant_4692', 'redundant_5325', 'redundant_3853', 'redundant_3849', 'redundant_2667', '216', 'redundant_5407', '565', '5513', '4646', '335', '816', '3379', 'redundant_PS_SCTC', '494', '1716', '413', '1715', 'redundant_PS_ERTC', 'redundant_910', 

In [ ]:
closeness_node_removal_and_calculate(G_adding_nodes_closeness)

Nodes to remove based on closeness centrality: ['811', 'redundant_811', '4451', 'redundant_4451', '4579', 'redundant_4579', '5407', 'redundant_5407', '4457', 'redundant_4457', 'PS_ARTC', 'redundant_PS_ARTC', '4434', 'redundant_4434', 'PS_DRDN', 'redundant_PS_DRDN', '3821', 'redundant_3821', '6067', 'redundant_6067', '736', 'redundant_736', 'PS_SNLS', 'redundant_PS_SNLS', 'PS_ERTC', 'redundant_PS_ERTC', '812', 'redundant_812', '3860', 'redundant_3860', '5696', 'redundant_5696', '4636', 'redundant_4636', '410', '408', 'redundant_410', 'redundant_408', '4450', 'redundant_4450', '1884', 'redundant_1884', '443', 'redundant_443', '409', 'redundant_409', '1599', 'redundant_1599', '5858', 'redundant_5858', '444', 'redundant_444', '5859', 'redundant_5859', '4581', 'redundant_4581', '5325', 'redundant_5325', '4435', 'redundant_4435', '446', 'redundant_446', '406', '3038', 'redundant_406', 'redundant_3038', '3864', 'redundant_3864', '3065', 'redundant_3065', '816', 'redundant_816', '1123', 'redun

In [ ]:
targeted_node_removal_and_calculate(G_adding_nodes_degree)

Nodes to remove based on degree centrality: ['PS_MBRT', 'PS_SCTC', 'PS_ERTC', 'PS_DRDN', '4692', 'PS_BERR', '439', 'PS_MVCT', 'PS_GILR', 'redundant_4692', 'PS_WITC', 'redundant_PS_ERTC', 'PS_PATC', 'PS_SNLS', 'redundant_439', '410', '413', 'PS_ARTC', 'redundant_PS_BERR', 'redundant_PS_DRDN', 'redundant_PS_SCTC', '335', '408', '443', '1715', '1716', '2523', '3468', 'PS_OHLN', '1124', '2526', '2535', '3358', '3926', 'PS_BAYP', 'redundant_PS_MBRT', 'redundant_PS_PATC', 'redundant_413', 'redundant_PS_SNLS', 'redundant_PS_WITC', '391', '445', '461', '494', '504', '565', '574', '660', '811', '1782', '1933', '1962', '2522', '3371', '3456', 'PS_LMTC', '4507', '4579', '4585', 'PS_TASM', 'PS_CHMP', '5513', 'redundant_PS_GILR', 'redundant_PS_ARTC', '347', '355', '403', '412', '444', '449', '512', '567', '586', '619', '643', '653', '661', '736', '864', '1123', '1884', '2364', '2604', '3202', '3271', '3379', '3417', '3821', '3844', '4338', '4578', 'PS_GISH', 'PS_WVTC', '5707', '20', '266', '330', '

In [ ]:
remove_random_nodes_and_calculate(G_adding_nodes)

Nodes removed: ['4422', '722', 'PS_GILR', '5797', '1889', '1587', '1391', '873', '5752', '673', 'PS_BORR', '5781', '3620', '593', '3915', '2852', '293', '272', '626', '1354', '1506', '3419', '4023', '245', '3694', '1207', '5595', '4485', '5427', '3619', '2835', '1372', '2997', '3910', '1908', 'redundant_3681', '53', '5962', 'redundant_2760', '968', 'PS_WITC', '2856', '2368', '1907', '949', '1331', '6001', '2345', '672', '621', '2601', '646', '2497', '2390', '4043', '1751', 'redundant_3524', '371', '5695', '3073', '3574', '798', '2589', '552', '3655', '2002', 'redundant_114', '4335', '4190', '2509', '3782', '1155', '5503', '496', '381', '4639', '1435', 'redundant_4434', '1965', '556', '1507', '666', 'redundant_2774', 'redundant_115', '3018', '4388', '2522', '982', '2545', '2468', '1285', 'PS_TAMN', '1818', '5459', '5006', '4466', '504', '4104', '4385', '1025', '3565', '5687', '1586', '985', '3088', '2599', '1832', '4424', '5059', '3675', '1366', '5029', '2246', '6054', 'redundant_653', 

In [ ]:
remove_random_nodes_and_calculate(G_adding_nodes_closeness)

Nodes removed: ['4422', '722', 'PS_GILR', '5797', '1889', '1587', '1391', '873', '5752', '673', 'PS_BORR', '5781', '3620', '593', '3915', '2852', '293', '272', '626', '1354', '1506', '3419', '4023', '245', '3694', '1207', '5595', '4485', '5427', '3619', '2835', '1372', '2997', '3910', '1908', 'redundant_1863', '53', '5962', 'redundant_3908', '968', 'PS_WITC', '2856', '2368', '1907', '949', '1331', '6001', '2345', '672', '621', '2601', '646', '2497', '2390', '4043', '1751', 'redundant_433', '371', '5695', '3073', '3574', '798', '2589', '552', '3655', '2002', 'redundant_3030', '4335', '4190', '2509', '3782', '1155', '5503', '496', '381', '4639', '1435', 'redundant_3821', '1965', '556', '1507', '666', 'redundant_578', 'redundant_2760', '3018', '4388', '2522', '982', '2545', '2468', '1285', 'PS_TAMN', '1818', '5459', '5006', '4466', '504', '4104', '4385', '1025', '3565', '5687', '1586', '985', '3088', '2599', '1832', '4424', '5059', '3675', '1366', '5029', '2246', '6054', 'redundant_1884',

In [ ]:
remove_random_nodes_and_calculate(G_adding_nodes_degree)

Nodes removed: ['4422', '722', 'PS_GILR', '5797', '1889', '1587', '1391', '873', '5752', '673', 'PS_BORR', '5781', '3620', '593', '3915', '2852', '293', '272', '626', '1354', '1506', '3419', '4023', '245', '3694', '1207', '5595', '4485', '5427', '3619', '2835', '1372', '2997', '3910', '1908', 'redundant_1938', '53', '5962', 'redundant_1110', '968', 'PS_WITC', '2856', '2368', '1907', '949', '1331', '6001', '2345', '672', '621', '2601', '646', '2497', '2390', '4043', '1751', 'redundant_1426', '371', '5695', '3073', '3574', '798', '2589', '552', '3655', '2002', 'redundant_4926', '4335', '4190', '2509', '3782', '1155', '5503', '496', '381', '4639', '1435', 'redundant_PS_GILR', '1965', '556', '1507', '666', 'redundant_113', 'redundant_247', '3018', '4388', '2522', '982', '2545', '2468', '1285', 'PS_TAMN', '1818', '5459', '5006', '4466', '504', '4104', '4385', '1025', '3565', '5687', '1586', '985', '3088', '2599', '1832', '4424', '5059', '3675', '1366', '5029', '2246', '6054', 'redundant_335

In [ ]:
#print the attributes of the nodes with the top ten closeness centrality
print("Top 10 nodes with the highest closeness centrality:")
for node, centrality in sorted(closeness.items(), key=lambda item: item[1], reverse=True)[:10]:
    print(f"Node: {node}, Closeness Centrality: {centrality}")
    print(f"Attributes: {G.nodes[node]}")
    print()

Top 10 nodes with the highest closeness centrality:
Node: 811, Closeness Centrality: 0.06868463551469987
Attributes: {'stop_name': 'Fruitdale & Southwest', 'lon': -121.916016, 'lat': 37.310795}

Node: 4451, Closeness Centrality: 0.06768286952276074
Attributes: {'stop_name': 'Deer Creek & Mid Arastradero', 'lon': -122.150143, 'lat': 37.395701}

Node: 4579, Closeness Centrality: 0.06711770879251022
Attributes: {'stop_name': 'Capitol & Story', 'lon': -121.826614, 'lat': 37.349775}

Node: 5407, Closeness Centrality: 0.06668641610600655
Attributes: {'stop_name': 'Parkmoor & Race', 'lon': -121.911535, 'lat': 37.316682}

Node: 4457, Closeness Centrality: 0.06668054646876541
Attributes: {'stop_name': 'Ohlone-Chynoweth Station (Bay 1)', 'lon': -121.86005, 'lat': 37.257349}

Node: PS_ARTC, Closeness Centrality: 0.06663802248038536
Attributes: {'stop_name': 'Alum Rock Transit Center', 'lon': -121.832557, 'lat': 37.35802}

Node: 4434, Closeness Centrality: 0.06615936585535978
Attributes: {'stop_na

In [ ]:
#print the attributes of the nodes with the top ten betweenness centrality
print("Top 10 nodes with the highest betweenness centrality:")
for node, centrality in sorted(betweenness.items(), key=lambda item: item[1], reverse=True)[:10]:
    print(f"Node: {node}, Betweenness Centrality: {centrality}")
    print(f"Attributes: {G.nodes[node]}")
    print()

Top 10 nodes with the highest betweenness centrality:
Node: 811, Betweenness Centrality: 0.19559866021164757
Attributes: {'stop_name': 'Fruitdale & Southwest', 'lon': -121.916016, 'lat': 37.310795}

Node: PS_MBRT, Betweenness Centrality: 0.13554273687816104
Attributes: {'stop_name': 'Milpitas Transit Center', 'lon': -121.891459, 'lat': 37.409955}

Node: PS_ARTC, Betweenness Centrality: 0.13169316279432003
Attributes: {'stop_name': 'Alum Rock Transit Center', 'lon': -121.832557, 'lat': 37.35802}

Node: PS_DRDN, Betweenness Centrality: 0.12790473751809794
Attributes: {'stop_name': 'San Jose Diridon Station', 'lon': -121.902394, 'lat': 37.330656}

Node: 4457, Betweenness Centrality: 0.1261801767145583
Attributes: {'stop_name': 'Ohlone-Chynoweth Station (Bay 1)', 'lon': -121.86005, 'lat': 37.257349}

Node: PS_SNLS, Betweenness Centrality: 0.11953707652341566
Attributes: {'stop_name': 'Snell Station', 'lon': -121.83062, 'lat': 37.248664}

Node: 4451, Betweenness Centrality: 0.11213037376758

In [ ]:
#print the attributes of the nodes with the top ten degree centrality
print("Top 10 nodes with the highest degree centrality:")
for node, centrality in sorted(degree.items(), key=lambda item: item[1], reverse=True)[:10]:
    print(f"Node: {node}, Degree Centrality: {centrality}")
    print(f"Attributes: {G.nodes[node]}")
    print()

Top 10 nodes with the highest degree centrality:
Node: PS_MBRT, Degree Centrality: 0.008552423186569527
Attributes: {'stop_name': 'Milpitas Transit Center', 'lon': -121.891459, 'lat': 37.409955}

Node: PS_ERTC, Degree Centrality: 0.006968641114982578
Attributes: {'stop_name': 'Eastridge Transit Center', 'lon': -121.810911, 'lat': 37.327411}

Node: PS_SCTC, Degree Centrality: 0.0063351282863477985
Attributes: {'stop_name': 'Santa Clara Transit Center', 'lon': -121.93735, 'lat': 37.352956}

Node: 4692, Degree Centrality: 0.005068102629078239
Attributes: {'stop_name': 'Great America ACE Amtrak Station', 'lon': -121.96689, 'lat': 37.406433}

Node: PS_BERR, Degree Centrality: 0.005068102629078239
Attributes: {'stop_name': 'Berryessa Transit Center', 'lon': -121.874769, 'lat': 37.369628}

Node: PS_DRDN, Degree Centrality: 0.004751346214760849
Attributes: {'stop_name': 'San Jose Diridon Station', 'lon': -121.902394, 'lat': 37.330656}

Node: 439, Degree Centrality: 0.004117833386126069
Attribu

In [ ]:
calculation(G)

Average Shortest Path Length: 23.035845062489667
Node Connectivity: 0
Size of Largest Connected Component: 3094
Global Efficiency: 0.05047551972717209
Diameter: 64
Clustering Coefficient: 0.02134335476640734
